In [ ]:
from samap.mapping import SAMAP
from samap.analysis import (get_mapping_scores, GenePairFinder,
                            sankey_plot, chord_plot, CellTypeTriangles, 
                            ParalogSubstitutions, FunctionalEnrichment,
                            convert_eggnog_to_homologs, GeneTriangles)
from samalg import SAM
import pandas as pd
import plotly.express as px
#NOTE:
#the package samalg is not called samalg but is instead called sam-algorithm
#Python 3.10 Works while 3.11+ does not

In [ ]:
A=pd.read_csv('chicken_and_mouse\chms\maps\chms\ch_to_ms.txt',sep='\t',index_col=0,header=None)
B=pd.read_csv('chicken_and_mouse\chms\maps\chms\ms_to_ch.txt',sep='\t',index_col=0,header=None)
A.head()

In [ ]:
B.head()

In [ ]:
#NOTE When loading in raw data, SAMap will automatically process them with SAM and save the results to an *_pr.h5ad file. For example, if the raw data is file.h5ad the processed data will be saved to file_pr.h5ad
fn1 = 'chicken_and_mouse\chms\chMG.h5ad' #chicken
fn2 = 'chicken_and_mouse\chms\mMG.h5ad' #mouse

In [ ]:
#NOTE: SAMap expects the above to be in a dictionary keyed by the species IDs determined in the BLAST step:
filenames = {'ch':fn1,'ms':fn2}
filenamesForLater = {'ch':fn1,'ms':fn2}

In [ ]:
#Load raw data mappings
sm = SAMAP(
        filenames,
        f_maps = 'chicken_and_mouse/chms/maps/',
        save_processed=True #if False, do not save the processed results to `*_pr.h5ad`
    )

In [ ]:
sm.run(pairwise=True)

In [ ]:
samap = sm.samap
#combine species, after this analysis is done, time for visualization
#The expression data in samap.adata is a block-diagonal matrix of each species' gene expressions.

In [ ]:
sm.scatter()

In [ ]:
#interactive plot
sm.samap.gui()
#requires modules colorlover, plotly, ipyevent

In [ ]:
#INVESTIGATION: Customize and label the clustering plot
#each species individually
sm.gui()

In [ ]:
#NOTES and Investigation For Analyzing/Exporting Samap Data:
#The species ID for each cell is stored in samap.adata.obs['species']
#The combined graph is stored in samap.adata.obsp['connectivities']
#The refined homology graph is stored in samap.adata.varp['homology_graph_reweighted'] (or sm.gnnm_refined) and the gene names corresponding to this graph are stored in samap.adata.var_names (or sm.gns)
#Print adata structure
print(dir(samap.adata))

In [ ]:
#save adata object as a pickle
import pickle
with open('samap_adata.pkl', 'wb') as file:
    pickle.dump(samap.adata, file)
with open('samap_raw_adata.pkl', 'wb') as file:
    pickle.dump(samap.adata_raw, file)

In [ ]:
#Pickle the entire samap object
import samap.utils
samap.utils.save_samap(sm, 'samap_object')

In [ ]:
#print entire object structure
print(dir(sm.samap))

In [ ]:
#print each species new file structure
print(filenames) 

In [ ]:
#save as h5ad
sm.samap.adata.write_h5ad(filename="Samap_h5ad.h5ad", compression=None, compression_opts=None, as_dense=())

In [ ]:
#save as loom, requires module loompy
import anndata as ad
sm.samap.adata.write_loom("samap_adata.loom", write_obsm_varm=True)